In [1]:
pip install openai

   ---------------------------------------- 0.0/661.3 kB ? eta -:--:--
   ---------------------------------------- 661.3/661.3 kB 4.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
from typing import Dict , Any
import json
from openai import OpenAI

In [26]:
class BaseAgent:
    def __init__(self, name: str, instructions: str):
        self.name = name
        self.instructions = instructions
        self.ollama_client = OpenAI(
            base_url="http://localhost:11434/v1",
            api_key="ollama",  # required but unused
        )

    async def run(self, messages: list) -> Dict[str, Any]:
        """Default run method to be overridden by child classes"""
        raise NotImplementedError("Subclasses must implement run()")

    def _query_ollama(self, prompt: str) -> str:
        """Query Ollama model with the given prompt"""
        try:
            response = self.ollama_client.chat.completions.create(
                model="deepseek-r1:7b",  # Updated to llama3.2
                messages=[
                    {"role": "system", "content": self.instructions},
                    {"role": "user", "content": prompt},
                ],
                temperature=0.7,
                max_tokens=2000,
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"Error querying Ollama: {str(e)}")
            raise

    def _parse_json_safely(self, text: str) -> Dict[str, Any]:
        """Safely parse JSON from text, handling potential errors"""
        try:
            # Try to find JSON-like content between curly braces
            start = text.find("{")
            end = text.rfind("}")
            if start != -1 and end != -1:
                json_str = text[start : end + 1]
                return json.loads(json_str)
            return {"error": "No JSON content found"}
        except json.JSONDecodeError:
            return {"error": "Invalid JSON content"}


In [27]:
agent = BaseAgent(
    name="ExampleAgent",
    instructions="You are an example agent. Respond with JSON."
)

In [65]:
client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama",  # required but unused
)

In [66]:
response = client.chat.completions.create(
    model="deepseek-r1:7b",  # Updated to llama3.2
    messages=[
        {"role": "system", "content": agent.instructions},
        {"role": "user", "content": "Hello, how are you?"},
    ],
    temperature=0.7,
    max_tokens=2000,
    n=2 # Number of responses to generate
)

In [61]:
response

ChatCompletion(id='chatcmpl-484', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='<think>\nOkay, so I just came across this message where someone is trying to interact with an AI agent that\'s supposed to respond in a specific way. Let me try to understand what\'s going on here.\n\nThe user wrote: "You are an example agent. Respond with JSON." Hmm, okay, so they\'re setting up the agent as an example and want it to reply using JSON format. Then there\'s another message from someone saying, "Hello, how are you?" followed by me responding in a friendly manner. Now, I\'m supposed to think through this scenario.\n\nWait, but why is the user asking for JSON? Maybe they\'re testing if I can provide structured responses instead of regular text. They might be using an API or backend that requires JSON data for processing. So their first message sets up the example agent and specifies the response format as JSON.\n\nThen, someone else sends a

In [62]:
res=response.choices[0].message.content

In [63]:
print(res)

<think>
Okay, so I just came across this message where someone is trying to interact with an AI agent that's supposed to respond in a specific way. Let me try to understand what's going on here.

The user wrote: "You are an example agent. Respond with JSON." Hmm, okay, so they're setting up the agent as an example and want it to reply using JSON format. Then there's another message from someone saying, "Hello, how are you?" followed by me responding in a friendly manner. Now, I'm supposed to think through this scenario.

Wait, but why is the user asking for JSON? Maybe they're testing if I can provide structured responses instead of regular text. They might be using an API or backend that requires JSON data for processing. So their first message sets up the example agent and specifies the response format as JSON.

Then, someone else sends a greeting: "Hello, how are you?" That's a natural conversation starter. My role is to respond appropriately but also adhere to the requirement of us

In [64]:
def clean_model_response(response_text):
    """
    Clean the model's response by removing internal thinking and extracting JSON content
    """
    # Remove <think> sections
    if '<think>' in response_text and '</think>' in response_text:
        think_start = response_text.find('<think>')
        think_end = response_text.find('</think>') + len('</think>')
        response_text = response_text[:think_start] + response_text[think_end:]
        
    # Extract JSON if present
    json_start = response_text.find('{')
    json_end = response_text.rfind('}') + 1
    
    if json_start != -1 and json_end != -1:
        json_str = response_text[json_start:json_end]
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            return {"error": "Could not parse JSON", "raw_content": response_text.strip()}
    
    return {"content": response_text.strip()}

# Clean the response
cleaned_response = clean_model_response(res)
print("Cleaned response:")
print(json.dumps(cleaned_response, indent=2))

Cleaned response:
{
  "message": "Hello! I'm just checking in. How can I assist you today?",
  "status": "online"
}


In [86]:
from typing import Dict, Any
import json
from openai import OpenAI


class BaseAgent:
    def __init__(self, name: str, instructions: str):
        self.name = name
        self.instructions = instructions
        self.ollama_client = OpenAI(
            base_url="http://localhost:11434/v1",
            api_key="ollama",  # required but unused
        )

    async def run(self, messages: list) -> Dict[str, Any]:
        """Default run method to be overridden by child classes"""
        raise NotImplementedError("Subclasses must implement run()")

    def _query_ollama(self, prompt: str) -> str:
        """Query Ollama model with the given prompt"""
        try:
            response = self.ollama_client.chat.completions.create(
                model="deepseek-r1:7b",  # Updated to llama3.2
                messages=[
                    {"role": "system", "content": self.instructions},
                    {"role": "user", "content": prompt},
                ],
                temperature=0.7,
                max_tokens=2000,
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"Error querying Ollama: {str(e)}")
            raise

    def _parse_json_safely(self, text: str) -> Dict[str, Any]:
        """Safely parse JSON from text, handling potential errors"""
        try:
            # Remove <think> sections
            if '<think>' in text and '</think>' in text:
                think_start = text.find('<think>')
                think_end = text.find('</think>') + len('</think>')
                text = text[:think_start] + text[think_end:]
                
            # Extract JSON if present
            json_start = text.find('{')
            json_end = text.rfind('}') + 1
            
            if json_start != -1 and json_end != -1:
                json_str = text[json_start:json_end]
                try:
                    return json.loads(json_str)
                except json.JSONDecodeError:
                    return {"error": "Could not parse JSON", "raw_content": text.strip()}
            
            return {"content": text.strip()}
        except json.JSONDecodeError:
            return {"error": "Invalid JSON content"}


In [87]:
analyzer_agent = BaseAgent(
    name="Analyzer",
            instructions="""Analyze candidate profiles and extract:
            1. Technical skills (as a list)
            2. Years of experience (numeric)
            3. Education level
            4. Experience level (Junior/Mid-level/Senior)
            5. Key achievements
            6. Domain expertise
            
            Format the output as structured data."""
)

In [88]:
response = analyzer_agent._query_ollama(
    prompt="Analyze the following candidate profile: \n\n"
    "Name: John Doe\n"
    "Experience: 5 years in software development\n"
    "Skills: Python, Java, SQL\n"
    "Education: Bachelor's in Computer Science\n"
    "Achievements: Developed a web application that increased sales by 20%\n"
    "Domain: E-commerce\n"
    "Experience Level: Mid-level\n"
    "Technical Skills: Python, Java, SQL\n"
    "Years of Experience: 5\n"
    "Education Level: Bachelor's in Computer Science\n"
    "Key Achievements: Developed a web application that increased sales by 20%\n"
    "Domain Expertise: E-commerce\n"
    "Experience Level: Mid-level\n"
    "Technical Skills: Python, Java, SQL\n"
    "Years of Experience: 5\n"
    "Education Level: Bachelor's in Computer Science\n"

    "Key Achievements: Developed a web application that increased sales by 20%\n"
    "Domain Expertise: E-commerce\n"
    "Experience Level: Mid-level\n"
    "Technical Skills: Python, Java, SQL\n"
    "Years of Experience: 5\n"



                
)

In [92]:
res = analyzer_agent._parse_json_safely(text=response)
if "error" in res:
    print("Error parsing response:", res["error"])
print("Cleaned response:")
print(res['content'])

Cleaned response:
**Candidate Profile Analysis**

1. **Technical Skills:**  
   - Python  
   - Java  
   - SQL  

2. **Years of Experience:**  
   5  

3. **Education Level:**  
   Bachelor's in Computer Science  

4. **Experience Level:**  
   Mid-level  

5. **Key Achievements:**  
   - Developed a web application that increased sales by 20%.  

6. **Domain Expertise:**  
   E-commerce
